# Data Collection Subject

How does the economy impact the popularity of movie generes?

# Scraping Data on Movies

In [130]:
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [131]:
url = 'https://www.the-numbers.com/market/genres'
html = requests.get(url).content
soup = BeautifulSoup(html,'lxml')

In [132]:
data = soup.find_all('table')[0].get_text()
newdata = data.split('\n')

for count in range(newdata.count('')):
    newdata.remove('')
    
newdata = newdata[1:]

genres=[]
for index in range(len(newdata)):
    if (index % 6)==1:
        item = newdata[index]
        genres.append(item)

In [424]:
newgenre=genres[:8]
for i in range(len(newgenre)):
    if newgenre[i] == 'Thriller/Suspense': 
        newgenre[i]= 'Thriller-or-Suspense'
    
    if newgenre[i] == 'Romantic Comedy':
        newgenre[i] = 'Romantic-Comedy'
    
    if newgenre[i] == 'Black Comedy':
        newgenre[i] = 'Black-Comedy'

In [425]:
#two dimensional list where dl[index] contains all of the market share datapoints by year for a genre
dl =[]
for genre in newgenre:
    #market share for each genre
    datalist=[]
    url = 'https://www.the-numbers.com/graphs/market/selected-item/genres/genre/gengres/'+genre+'/'+genre+'#_ABSTRACT_RENDERED_ID_0'
    html = requests.get(url).content
    soup = BeautifulSoup(html,'lxml')
    soup.find_all('script')[1].get_text()
    
    text =soup.find_all('script')[1].get_text()
    newtext = text.split(';')
    twotext = newtext[2].split('\t')

    for count in range(twotext.count('')):
        twotext.remove('')

    threetext = twotext[3][1:].split(', ')


    for index in range(len(threetext)):
        data = threetext[index].split(',')
        year=data[0][2:6]
        datapoint=float(data[2][:4])
        datalist.append(datapoint/100)
    
    dl.append(datalist)

#number of movies produced
dlnummade=[]
for genre in newgenre:
    url='https://www.the-numbers.com/market/genre/'+genre
    html = requests.get(url).content
    soup = BeautifulSoup(html,'lxml')
    p2text = soup.find_all('table')[1].get_text()
    makesomepoints= p2text.split('\n')

    for x in range(makesomepoints.count('')):
        makesomepoints.remove('')

    tabledata = makesomepoints[1:]
    pointsingenre=[]

    for s in range(len(tabledata)):
        if s%8==1:
            pointsingenre.append(tabledata[s])
    dlnummade.append(pointsingenre)

In [427]:
newdata=pd.DataFrame()
#append market share
index = 0
for thegenre in newgenre:
    if len(dl[index])!=26:
        dl[index].insert(0,0)
    if len(dlnummade[index])!=26:
        dlnummade[index].insert(0,0)
    newdata[thegenre + ' Market Share']=dl[index]
    newdata[thegenre + ' Movies Made'] = dlnummade[index]
    index += 1

if len(newdata)!=24:
    newdata = newdata.iloc[:-2]
gdp = pd.read_csv("gdp.csv")[(1995-1961):]
gdp.reset_index(inplace=True)
newdata['GDP % Growth']= gdp[' GDP Growth (%)']
newdata['GDP Annual Change']= gdp[' Annual Change']


pce = pd.read_csv("pce.csv")
pcedata=[]
for index in range(len(pce.loc[0])):
    pcedata.append(pce.iloc[0][index])
newdata['PCE'] = pcedata[2:]

final_data = newdata.rename(index = lambda s: s + 1995)
final_data

,Adventure Market Share,Adventure Movies Made,Action Market Share,Action Movies Made,Drama Market Share,Drama Movies Made,Comedy Market Share,Comedy Movies Made,Thriller-or-Suspense Market Share,Thriller-or-Suspense Movies Made,Horror Market Share,Horror Movies Made,Romantic-Comedy Market Share,Romantic-Comedy Movies Made,Musical Market Share,Musical Movies Made,GDP % Growth,GDP Annual Change,PCE
1995,0.145,28,0.197,29,0.287,131,0.1970,61,0.0621,20,0.0278,15,0.0559,16,0.0000,0,2.6843,-1.34,71.811
1996,0.223,33,0.211,36,0.234,121,0.1800,70,0.0531,23,0.0195,9,0.0740,19,0.0004,2,3.7725,1.09,73.346
1997,0.138,24,0.229,36,0.188,120,0.2040,73,0.1010,23,0.0641,11,0.0467,22,0.0179,5,4.4472,0.67,74.623
1998,0.238,29,0.143,30,0.232,130,0.1160,74,0.1160,22,0.0470,16,0.0739,14,0.0099,8,4.4814,0.03,75.216
1999,0.219,27,0.124,34,0.199,158,0.1840,79,0.0783,17,0.0650,16,0.0984,28,0.0032,11,4.7532,0.27,76.338
2000,0.176,30,0.168,31,0.232,170,0.1960,80,0.0759,27,0.0491,14,0.0497,23,0.0123,10,4.1275,-0.63,78.235
2001,0.271,32,0.166,32,0.164,154,0.1760,90,0.0807,26,0.0468,16,0.0633,24,0.0083,7,0.9983,-3.13,79.738
2002,0.307,46,0.152,32,0.174,179,0.1420,87,0.1030,33,0.0325,12,0.0627,24,0.0063,10,1.7417,0.74,80.789
2003,0.263,47,0.229,38,0.120,182,0.1950,81,0.0416,21,0.0490,17,0.0542,26,0.0184,6,2.8612,1.12,82.358
2004,0.310,48,0.115,28,0.148,231,0.1800,121,0.0853,29,0.0510,22,0.0701,33,0.0024,5,3.7989,0.94,84.411


# Potential Problems with the Dataset

This data has some limitations that should be taken into consideration when analyzing the results. Firstly, this data is restricted to the popularity of "box office" movies and does not consider other mediums for watching movies. Therefore, even though a certain genre may be popular in theaters, another type of genre may be trending through other mediums such as streaming services and DVD rentals/purchases. However, movies were a popular medium until more recently, so by going back to 1995 in terms of research collection, we can mitigate this issue.

Additionally, this data is limited to the period of 1995-2018. Therefore, it takes into account only one major economic downturn, the Great Recession, and is therefore a small sample size when considering the impact of the economy on movie preferences. This data may demonstrate correlation, but it is difficult to state causation becuase a genre may simply be trending at the time due to other external factors. Although it is hard to state causation, it will still be interesting to see if a correlation does exist. Maybe people prefer horror and are more pessimistic when the economy is down!

A third limitation is that the medium through which users engage with a genre varies by genre. For example, users that enjoy musicals are more likely to go to Broadway than movie theaters. However, by restricting our question to movie popularity, we can minimize the impact of user medium preferences.

Finally, this data is for the United States, but there could be potential skews in popularity based on metropolitan versus rural considerations. More people might prefer watching horror, but since movie theaters are more accessbile in metropolitan areas, popularity might be skewed towards urban area preferences.

# Data Description Draft

### What are the observations (rows) and the attributes (columns)?

** Observations: ** 

*Movie Popularity*

* Market share for the 8 most popular film genres: adventure, action, drama, comedy, thriller, horror, romantic comedy, and musicals. This will enable us to figure out which is the most sold in proportion to the total market. 

* Additionally, we noted the number of movies made, so that we can normalize market share percentage by production percentage. If only 2 movies are made in a genre but it caputes 20% of the market share, this is more noticible than a genre with a 100 movies made and the same market share.

*State of the Economy*

* We use rate of GDP growth and change of rate of GDP to measure the state of the economy. GDP is generally always increasing, so the rate of change provides a more detailed view of the economy.

* We use Personal Consupmtion Expenditure (PCE) to determine how much consumers are willing to spend. We were orginally going to use the Consumer Pricing Index (CPI), but PCE is less impacted by inflation and incorporates all household expenses, not just out of pocket expenses.

** Attributes: ** 

* The years from 1995 to 2018.

### Why was this dataset created?

This dataset is a combination of data scaping and csv files.

** Movie Data **
* The data scraping comes the Numbers, with the following About Us description: *'The Numbers was officially launched on October 17th, 1997 as a free resource for industry professionals, the investment community, and movie fans to track business information on movies.'*

** Economic Data **
* The information on the economy comes from the U.S. Bureau of Economic Analysis (BEA). The government uses this information to keep track of major economic indicators and figure out how to optimize the economy to increase citizen standard of living.

### Who funded the creation of the dataset?

** Movie Data **
* Nash Information Service: a movie industry consulting service

** Economic Data **
* The U.S. Government

### What processes might have influenced what data was observed and recorded and what was not?
**Movie Data**
* This data is recorded by ticket sales. The movie industry accurately records this data because it helps it evaluate performance.
* Another source of influence includes the genres in which the movies are sorted. A movie could fall under two different categories and attract two different audiences, but for simplification, were placed into one category, probably based on which the movie "most" fell under.

**U.S. Government**
* This data is recored by using "bundles" and putting weights on each item in the bundle. Therefore, decisions on the bundling and the weight provided would impact what was recorded.

### What preprocessing was done, and how did the data come to be in the form that you are using?
**Movie Data**
* This data was found by first finding the number of tickets sold per movie and the category under which each movie falls. Then the labels and the sums for each genre were made accordingly. Then finally, the percentages per genre were found.

**Economic Data**
* This data was probably collected by finding industry market size and multiplying it by the percentage weights associated with the PCE. 

### If people are involved, were they aware of the data collection and if so, what purpose did they expect the data to be used for?
People were probably not involved. Much of the daata was probably collected on a transaction basis.

### Where can your raw source data be found, if applicable? Provide a link to the raw data (hosted in a Cornell Google Drive​ or ​Cornell Box)​? 

Data Scraping: https://www.the-numbers.com/market/genres

Data Collection: https://www.bea.gov/data/gdp/gross-domestic-product
